### BRSET

* [Dataset](https://physionet.org/content/brazilian-ophthalmological/1.0.0/)

* [Original Paper](https://www.medrxiv.org/content/10.1101/2024.01.23.24301660v1)

In [1]:
%cd ..

/home/datascience/Data Fusion


### Setup Environment:

In [2]:
import os
import pandas as pd

from src.classifiers import process_labels, split_data
from src.classifiers_base import preprocess_df

from transformers import BertTokenizer

from src.multimodal_data_loader import VQADataset
from torch.utils.data import DataLoader

from src.classifiers_base_cpu_metrics import calculate_memory

In [3]:
PATH = 'datasets/brset/'

In [4]:
text_path = os.path.join(PATH, 'labels.csv')
images_path = os.path.join(PATH, 'images')

## Get data

In [5]:
df = pd.read_csv(text_path)
df

,image_id,DR_ICDR,text,DR_2,DR_3,split
0,img00001,0,"An image from the right eye of a male patient,...",0,0,train
1,img00002,0,"An image from the left eye of a male patient, ...",0,0,test
2,img00003,0,An image from the right eye of a female patien...,0,0,train
3,img00004,0,An image from the left eye of a female patient...,0,0,train
4,img00005,0,"An image from the right eye of a male patient,...",0,0,test
...,...,...,...,...,...,...
16261,img16262,1,"An image from the left eye of a male patient, ...",1,1,test
16262,img16263,0,"An image from the right eye of a male patient,...",0,0,train
16263,img16264,0,"An image from the left eye of a male patient, ...",0,0,test
16264,img16265,0,"An image from the right eye of a male patient,...",0,0,train


## Data Perparation

In [6]:
# Select features and labels vectors
text_columns = 'text'
image_columns = 'image_id'
label_columns = 'DR_2'

df = preprocess_df(df, image_columns, images_path)

# Split the data
train_df, test_df = split_data(df)

# Process and one-hot encode labels for training set
train_labels, mlb, train_columns = process_labels(train_df, col=label_columns)
test_labels = process_labels(test_df, col=label_columns, train_columns=train_columns)

100%|██████████| 16266/16266 [00:24<00:00, 658.89it/s]


Train Shape: (13012, 6)
Test Shape: (3254, 6)


In [7]:
train_df

,image_id,DR_ICDR,text,DR_2,DR_3,split
0,datasets/brset/images/img00001.jpg,0,"An image from the right eye of a male patient,...",0,0,train
2,datasets/brset/images/img00003.jpg,0,An image from the right eye of a female patien...,0,0,train
3,datasets/brset/images/img00004.jpg,0,An image from the left eye of a female patient...,0,0,train
6,datasets/brset/images/img00007.jpg,0,"An image from the right eye of a male patient,...",0,0,train
7,datasets/brset/images/img00008.jpg,0,"An image from the left eye of a male patient, ...",0,0,train
...,...,...,...,...,...,...
16259,datasets/brset/images/img16260.jpg,0,An image from the left eye of a female patient...,0,0,train
16260,datasets/brset/images/img16261.jpg,1,"An image from the right eye of a male patient,...",1,1,train
16262,datasets/brset/images/img16263.jpg,0,"An image from the right eye of a male patient,...",0,0,train
16264,datasets/brset/images/img16265.jpg,0,"An image from the right eye of a male patient,...",0,0,train


In [8]:
# Instantiate tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
train_dataset = VQADataset(train_df, text_columns, image_columns, label_columns, mlb, train_columns, tokenizer)
test_dataset = VQADataset(test_df, text_columns, image_columns, label_columns, mlb, train_columns, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

### Models

In [10]:
if label_columns == 'DR_2':
    output_size = 1
else:
    output_size = len(pd.unique(train_df[label_columns]))
multilabel = False

In [11]:
calculate_memory(train_loader, test_loader, output_size)

Early fusion:
Average Memory per Batch in Train: 36.63 MB
Total Memory Usage per Epoch Train: 7471.78 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 9.16 MB
Total Memory Usage per Epoch Test: 467.13 MB (excluding model parameters)
Model: 
Model Memory Usage: 747.94 MB

Late fusion:
Average Memory per Batch in Train: 36.63 MB
Total Memory Usage per Epoch Train: 7471.78 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 9.16 MB
Total Memory Usage per Epoch Test: 467.13 MB (excluding model parameters)
Model: 
Model Memory Usage: 747.57 MB
